## Importing data : UNSW-NB15_1.csv
[UNSW-NB15 Dataset Description](https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/)

My suggestion would be directly download the file to your computer or save it to google drive then mount it to colab.

In [2]:
import pandas as pd

filepath = '/content/drive/My Drive/UNSW-NB15_1.csv'
unsw_df = pd.read_csv(filepath)
unsw_df.columns = ['srcip','sport','dstip','dsport','proto','state','dur','sbytes','dbytes','sttl','dttl','sloss','dloss',
                   'service','Sload','Dload','Spkts','Dpkts','swin','dwin','stcpb','dtcpb','smeansz','dmeansz','trans_depth',
                   'res_bdy_len','Sjit','Djit','Stime','Ltime','Sintpkt','Dintpkt','tcprtt','synack','ackdat','is_sm_ips_ports',
                   'ct_state_ttl','ct_flw_http_mthd','is_ftp_login','ct_ftp_cmd','ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ ltm',
                   'ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','attack_cat','label']

unsw_df.head(5)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,3,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,NaN,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,12,8,1,2,2,1,1,NaN,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0,0,0,6,9,1,1,1,1,1,NaN,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0,0,0,7,9,1,1,1,1,1,NaN,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,29,0,0,-,43503.23438,23896.14258,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003,24.315000,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,NaN,0


---
## Exploring data

In [3]:
unsw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 49 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   srcip             700000 non-null  object 
 1   sport             700000 non-null  object 
 2   dstip             700000 non-null  object 
 3   dsport            700000 non-null  object 
 4   proto             700000 non-null  object 
 5   state             700000 non-null  object 
 6   dur               700000 non-null  float64
 7   sbytes            700000 non-null  int64  
 8   dbytes            700000 non-null  int64  
 9   sttl              700000 non-null  int64  
 10  dttl              700000 non-null  int64  
 11  sloss             700000 non-null  int64  
 12  dloss             700000 non-null  int64  
 13  service           700000 non-null  object 
 14  Sload             700000 non-null  float64
 15  Dload             700000 non-null  float64
 16  Spkts             70

In [4]:
attack_df = unsw_df[unsw_df['label']==1]
print('{} out of 700000 are attacks.\nThat is to say, {}% of the data.'.format(len(unsw_df[unsw_df['label']==1].index),
                                                                               round(len(attack_df.index)/len(unsw_df.index)*100, 3)))

22215 out of 700000 are attacks.
That is to say, 3.174% of the data.


In [5]:
# Downsample the normal ones since the amount of normals are much larger than the amount of attacks.

normal_df = unsw_df[unsw_df['label']==0]
normal_df = normal_df.sample(30000)
normal_df.shape

(30000, 49)

In [10]:
sample_df = pd.concat([normal_df, attack_df])
sample_df = sample_df.sample(frac = 1) # shuffle rows
sample_df.shape

(52215, 49)

---
##Preprocessing / Feature engineering

### attack categories
there are 9 types of attack for all attacks, thus we have 10 categories for this dataset (9 + normal).

In [11]:
attack_df['attack_cat'].value_counts()

Generic           7522
Exploits          5409
 Fuzzers          5051
Reconnaissance    1759
DoS               1167
Backdoors          534
Analysis           526
Shellcode          223
Worms               24
Name: attack_cat, dtype: int64

In [12]:
sample_df['attack_cat'].fillna(value='normal', inplace=True)
sample_df['attack_cat'].value_counts()

normal            30000
Generic            7522
Exploits           5409
 Fuzzers           5051
Reconnaissance     1759
DoS                1167
Backdoors           534
Analysis            526
Shellcode           223
Worms                24
Name: attack_cat, dtype: int64

In [13]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
sample_df['attack_cat'] = le.fit_transform(sample_df['attack_cat'])
sample_df['attack_cat'].value_counts()

9    30000
5     7522
4     5409
0     5051
6     1759
3     1167
2      534
1      526
7      223
8       24
Name: attack_cat, dtype: int64

### protocols, states and services
convert the only 3 categorical columns by using one-hot encoding.

In [14]:
sample_df['proto'].value_counts()

tcp           30999
udp           17320
unas           1442
ospf            294
arp             161
              ...  
nsfnet-igp       14
pup              14
pvp              14
iplt             14
icmp              7
Name: proto, Length: 131, dtype: int64

In [15]:
proto_onehot_df = pd.get_dummies(sample_df['proto'])
proto_onehot_df

,3pc,a/n,aes-sp3-d,any,argus,aris,arp,ax.25,bbn-rcc,bna,br-sat-mon,cbt,cftp,chaos,compaq-peer,cphb,cpnx,crtp,crudp,dcn,ddp,ddx,dgp,egp,eigrp,emcon,encap,etherip,fc,fire,ggp,gmtp,gre,hmp,i-nlsp,iatp,ib,icmp,idpr,idpr-cmtp,...,sat-expak,sat-mon,sccopmce,scps,sctp,sdrp,secure-vmtp,sep,skip,sm,smp,snp,sprite-rpc,sps,srp,st2,stp,sun-nd,swipe,tcf,tcp,tlsp,tp++,trunk-1,trunk-2,ttp,udp,unas,uti,vines,visa,vmtp,vrrp,wb-expak,wb-mon,wsn,xnet,xns-idp,xtp,zero
87823,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
417228,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
363125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
145077,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
436433,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333563,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
556504,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
554685,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
131776,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
sample_df['state'].value_counts()

FIN    30653
INT    12928
CON     8468
REQ      154
URH        5
CLO        3
ACC        2
ECR        1
ECO        1
Name: state, dtype: int64

In [17]:
state_onehot_df = pd.get_dummies(sample_df['state'])
state_onehot_df

,ACC,CLO,CON,ECO,ECR,FIN,INT,REQ,URH
87823,0,0,0,0,0,0,1,0,0
417228,0,0,0,0,0,1,0,0,0
363125,0,0,1,0,0,0,0,0,0
145077,0,0,0,0,0,0,1,0,0
436433,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
333563,0,0,0,0,0,1,0,0,0
556504,0,0,1,0,0,0,0,0,0
554685,0,0,0,0,0,1,0,0,0
131776,0,0,0,0,0,0,1,0,0


In [18]:
sample_df['service'].value_counts()

-           30297
dns         12047
http         4330
ftp-data     1793
smtp         1626
ftp          1207
ssh           660
pop3          206
ssl            20
snmp           14
dhcp            7
radius          6
irc             2
Name: service, dtype: int64

In [19]:
service_onehot_df = pd.get_dummies(sample_df['service'])
service_onehot_df

,-,dhcp,dns,ftp,ftp-data,http,irc,pop3,radius,smtp,snmp,ssh,ssl
87823,1,0,0,0,0,0,0,0,0,0,0,0,0
417228,1,0,0,0,0,0,0,0,0,0,0,0,0
363125,0,0,1,0,0,0,0,0,0,0,0,0,0
145077,0,0,1,0,0,0,0,0,0,0,0,0,0
436433,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333563,0,0,0,0,1,0,0,0,0,0,0,0,0
556504,0,0,1,0,0,0,0,0,0,0,0,0,0
554685,1,0,0,0,0,0,0,0,0,0,0,0,0
131776,0,0,1,0,0,0,0,0,0,0,0,0,0


### IP conversion

In [20]:
ip_df = sample_df[['srcip','dstip','label']]
print(ip_df.shape)
ip_df.head()

(52215, 3)


,srcip,dstip,label
87823,175.45.176.1,149.171.126.12,1
417228,59.166.0.2,149.171.126.1,0
363125,59.166.0.9,149.171.126.6,0
145077,175.45.176.1,149.171.126.18,1
436433,59.166.0.4,149.171.126.4,0


In [21]:
group = ip_df.groupby('srcip')
group.size()

srcip
10.40.170.2           40
10.40.182.1           75
10.40.182.3           38
10.40.85.1            66
10.40.85.30           32
149.171.126.0          9
149.171.126.1         11
149.171.126.11         1
149.171.126.18       266
149.171.126.19         1
149.171.126.2         15
149.171.126.3         12
149.171.126.4         10
149.171.126.5          9
149.171.126.6         10
149.171.126.7         10
149.171.126.8          7
149.171.126.9          5
175.45.176.0        3359
175.45.176.1       13296
175.45.176.2        2640
175.45.176.3        3138
192.168.241.243        5
59.166.0.0          2999
59.166.0.1          2915
59.166.0.2          2983
59.166.0.3          2907
59.166.0.4          2956
59.166.0.5          3046
59.166.0.6          2889
59.166.0.7          2744
59.166.0.8          2864
59.166.0.9          2857
dtype: int64

In [22]:
group = ip_df.groupby('dstip')
group.size()

dstip
10.40.170.2          40
10.40.182.3          68
10.40.85.1           32
10.40.85.30          21
149.171.126.0      2978
149.171.126.1      3006
149.171.126.10     1291
149.171.126.11     1411
149.171.126.12     4486
149.171.126.13     1411
149.171.126.14     1351
149.171.126.15     1282
149.171.126.16      992
149.171.126.17     1256
149.171.126.18     7975
149.171.126.19      978
149.171.126.2      2959
149.171.126.3      2954
149.171.126.4      2934
149.171.126.5      2950
149.171.126.6      2853
149.171.126.7      2791
149.171.126.8      2838
149.171.126.9      2897
175.45.176.0          3
175.45.176.1        265
192.168.241.243       5
224.0.0.5            90
59.166.0.0            8
59.166.0.1           14
59.166.0.2            9
59.166.0.3            6
59.166.0.4           12
59.166.0.5            6
59.166.0.6            8
59.166.0.7            6
59.166.0.8           12
59.166.0.9           17
dtype: int64

In [23]:
#sources of attacks
ip_df[ip_df['label']==1]['srcip'].value_counts()

175.45.176.1    13251
175.45.176.0     3293
175.45.176.3     3057
175.45.176.2     2614
Name: srcip, dtype: int64

In [24]:
# victim IPs
ip_df[ip_df['label']==1]['dstip'].value_counts()

149.171.126.18    7962
149.171.126.12    4478
149.171.126.11    1392
149.171.126.13    1371
149.171.126.14    1324
149.171.126.15    1272
149.171.126.10    1261
149.171.126.17    1217
149.171.126.16     979
149.171.126.19     959
Name: dstip, dtype: int64

---
The attacks are all from the IPs in 175.45.176.0/30 sent to IPs in 149.171.126.0/27. 

Converting IP into categories of network might be a feasible solution.

---

In [25]:
# grouping srcip

for i in range(len(ip_df.index)):
  if ip_df.iloc[i]['srcip'].startswith('175.45.176'):
    ip_df.iloc[i, ip_df.columns.get_loc('srcip')] = 'S175.45.176.0/24'
  elif ip_df.iloc[i]['srcip'].startswith('59.166.0'):
    ip_df.iloc[i, ip_df.columns.get_loc('srcip')] = 'S59.166.0.0/24'
  elif ip_df.iloc[i]['srcip'].startswith('149.171.126'):
    ip_df.iloc[i, ip_df.columns.get_loc('srcip')] = 'S149.171.126.0/24'
  elif ip_df.iloc[i]['srcip'].startswith('10.40'):
    ip_df.iloc[i, ip_df.columns.get_loc('srcip')] = 'S10.40.0.0/16'
  else:
    ip_df.iloc[i, ip_df.columns.get_loc('srcip')] = 'Sothers'

ip_df['srcip'].value_counts()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


S59.166.0.0/24       29160
S175.45.176.0/24     22433
S149.171.126.0/24      366
S10.40.0.0/16          251
Sothers                  5
Name: srcip, dtype: int64

In [26]:
# grouping dstip

for i in range(len(ip_df.index)):
  if ip_df.iloc[i]['dstip'].startswith('175.45.176'):
    ip_df.iloc[i, ip_df.columns.get_loc('dstip')] = 'D175.45.176.0/24'
  elif ip_df.iloc[i]['dstip'].startswith('59.166.0'):
    ip_df.iloc[i, ip_df.columns.get_loc('dstip')] = 'D59.166.0.0/24'
  elif ip_df.iloc[i]['dstip'].startswith('149.171.126'):
    ip_df.iloc[i, ip_df.columns.get_loc('dstip')] = 'D149.171.126.0/24'
  elif ip_df.iloc[i]['dstip'].startswith('10.40'):
    ip_df.iloc[i, ip_df.columns.get_loc('dstip')] = 'D10.40.0.0/16'
  else:
    ip_df.iloc[i, ip_df.columns.get_loc('dstip')] = 'Dothers'

ip_df['dstip'].value_counts()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


D149.171.126.0/24    51593
D175.45.176.0/24       268
D10.40.0.0/16          161
D59.166.0.0/24          98
Dothers                 95
Name: dstip, dtype: int64

In [27]:
ip_df.head()

,srcip,dstip,label
87823,S175.45.176.0/24,D149.171.126.0/24,1
417228,S59.166.0.0/24,D149.171.126.0/24,0
363125,S59.166.0.0/24,D149.171.126.0/24,0
145077,S175.45.176.0/24,D149.171.126.0/24,1
436433,S59.166.0.0/24,D149.171.126.0/24,0


In [28]:
ip_onehot_df = pd.concat([pd.get_dummies(ip_df['srcip']), pd.get_dummies(ip_df['dstip'])], axis=1)
ip_onehot_df

,S10.40.0.0/16,S149.171.126.0/24,S175.45.176.0/24,S59.166.0.0/24,Sothers,D10.40.0.0/16,D149.171.126.0/24,D175.45.176.0/24,D59.166.0.0/24,Dothers
87823,0,0,1,0,0,0,1,0,0,0
417228,0,0,0,1,0,0,1,0,0,0
363125,0,0,0,1,0,0,1,0,0,0
145077,0,0,1,0,0,0,1,0,0,0
436433,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
333563,0,0,0,1,0,0,1,0,0,0
556504,0,0,0,1,0,0,1,0,0,0
554685,0,0,0,1,0,0,1,0,0,0
131776,0,0,1,0,0,0,1,0,0,0


### record start time / record last time

In [29]:
sample_df['last-start'] = sample_df['Ltime'] - sample_df['Stime']
sample_df = sample_df.drop(['Stime','Ltime'], axis=1)
sample_df.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label,last-start
87823,175.45.176.1,0,149.171.126.12,0,unas,INT,0.000011,200,0,254,0,0,0,-,7.272727e+07,0.000000,2,0,0,0,0,0,100,0,0,0,0.000000,0.000000,0.01100,0.000000,0.000000,0.000000,0.000000,0,2,0,0,0,3,3,2,6,2,2,2,2,1,0
417228,59.166.0.2,40282,149.171.126.1,5190,tcp,FIN,0.046501,1140,2366,31,29,4,4,-,1.821896e+05,377970.375000,14,14,255,255,3597739155,1448389720,81,169,0,0,285.080705,8.072192,3.54900,3.536308,0.000668,0.000525,0.000143,0,0,0,0,0,19,25,22,19,4,4,4,9,0,0
363125,59.166.0.9,36444,149.171.126.6,53,udp,CON,0.001057,146,178,31,29,0,0,dns,5.525071e+05,673604.500000,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,0.00500,0.008000,0.000000,0.000000,0.000000,0,0,0,0,0,3,4,5,1,1,1,1,9,0,0
145077,175.45.176.1,47439,149.171.126.18,53,udp,INT,0.000005,114,0,254,0,0,0,dns,9.120000e+07,0.000000,2,0,0,0,0,0,57,0,0,0,0.000000,0.000000,0.00500,0.000000,0.000000,0.000000,0.000000,0,2,0,0,0,7,7,4,6,4,4,4,5,1,0
436433,59.166.0.4,42511,149.171.126.4,57561,tcp,FIN,0.254459,8928,320,31,29,4,1,ftp-data,2.606628e+05,8394.280273,14,6,255,255,2869564621,931176653,638,53,0,0,1591.844621,99.560859,19.57377,50.050000,0.001766,0.001602,0.000164,0,0,0,0,0,3,3,22,15,3,3,12,9,0,0


### port

In [30]:
sample_df.sport.value_counts()

0        3136
47439    2766
1043     2092
47439    1342
1043      987
         ... 
34931       1
57484       1
57480       1
34940       1
32895       1
Name: sport, Length: 31680, dtype: int64

In [31]:
sample_df.dsport.value_counts()

53       6937
53       5836
0        3572
80       3328
80       2609
         ... 
12161       1
4758        1
60021       1
26997       1
60913       1
Name: dsport, Length: 10906, dtype: int64

Too many ports used in this dataset, in this case I dropped these 2 columns since I can't think of a better way to analyze them so far.

In [32]:
sample_df = sample_df.drop(['sport','dsport'], axis=1)
sample_df.shape

(52215, 46)

### data normalization
Transform all numerical features by minmax scaling.

In [33]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52215 entries, 87823 to 139275
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   srcip             52215 non-null  object 
 1   dstip             52215 non-null  object 
 2   proto             52215 non-null  object 
 3   state             52215 non-null  object 
 4   dur               52215 non-null  float64
 5   sbytes            52215 non-null  int64  
 6   dbytes            52215 non-null  int64  
 7   sttl              52215 non-null  int64  
 8   dttl              52215 non-null  int64  
 9   sloss             52215 non-null  int64  
 10  dloss             52215 non-null  int64  
 11  service           52215 non-null  object 
 12  Sload             52215 non-null  float64
 13  Dload             52215 non-null  float64
 14  Spkts             52215 non-null  int64  
 15  Dpkts             52215 non-null  int64  
 16  swin              52215 non-null  i

In [34]:
cols_to_normalize = ['dur','sbytes','dbytes','sttl','dttl','sloss','dloss','Sload','Dload','Spkts','Dpkts',
                     'swin','dwin','stcpb','dtcpb','smeansz','dmeansz','trans_depth','res_bdy_len','Sjit','Djit',
                     'Sintpkt','Dintpkt','tcprtt','synack','ackdat','is_sm_ips_ports','ct_state_ttl',
                     'ct_flw_http_mthd','is_ftp_login','ct_ftp_cmd','ct_srv_src','ct_srv_dst','ct_dst_ltm',
                     'ct_src_ ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','last-start']
sample_df[cols_to_normalize] = sample_df[cols_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

sample_df.head()

,srcip,dstip,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label,last-start
87823,175.45.176.1,149.171.126.12,unas,INT,1.833348e-07,0.000015,0.000000,0.996078,0.000000,0.000000,0.000000,-,0.013805,0.000000,0.000196,0.000000,0.0,0.0,0.000000,0.000000,0.066489,0.000000,0.0,0.0,0.000000,0.000000,1.833212e-07,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.0,0.046512,0.048780,0.024390,0.102041,0.028571,0.030303,0.027027,2,1,0.0
417228,59.166.0.2,149.171.126.1,tcp,FIN,7.750228e-04,0.000083,0.000161,0.121569,0.114173,0.000785,0.000730,-,0.000035,0.017908,0.001373,0.001276,1.0,1.0,0.837667,0.337233,0.053856,0.114888,0.0,0.0,0.000248,0.000012,5.914609e-05,7.078510e-05,0.000546,0.000882,0.000151,0.0,0.000000,0.0,0.0,0.0,0.418605,0.585366,0.512195,0.367347,0.085714,0.090909,0.081081,9,0,0.0
363125,59.166.0.9,149.171.126.6,udp,CON,1.761681e-05,0.000011,0.000012,0.121569,0.114173,0.000000,0.000000,dns,0.000105,0.031915,0.000196,0.000182,0.0,0.0,0.000000,0.000000,0.048537,0.060503,0.0,0.0,0.000000,0.000000,8.332782e-08,1.601333e-07,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.046512,0.073171,0.097561,0.000000,0.000000,0.000000,0.000000,9,0,0.0
145077,175.45.176.1,149.171.126.18,udp,INT,8.333399e-08,0.000008,0.000000,0.996078,0.000000,0.000000,0.000000,dns,0.017312,0.000000,0.000196,0.000000,0.0,0.0,0.000000,0.000000,0.037899,0.000000,0.0,0.0,0.000000,0.000000,8.332782e-08,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.0,0.139535,0.146341,0.073171,0.102041,0.085714,0.090909,0.081081,5,1,0.0
436433,59.166.0.4,149.171.126.4,tcp,FIN,4.241017e-03,0.000653,0.000022,0.121569,0.114173,0.000785,0.000182,ftp-data,0.000049,0.000398,0.001373,0.000547,1.0,1.0,0.668125,0.216809,0.424202,0.036030,0.0,0.0,0.001384,0.000145,3.262079e-04,1.001834e-03,0.001445,0.002692,0.000174,0.0,0.000000,0.0,0.0,0.0,0.046512,0.048780,0.512195,0.285714,0.057143,0.060606,0.297297,9,0,0.0


### merge converted dataframes

Now we have 4 features converted into one-hot encoding dataframes (proto, state, service and ip). 

Simply merge them into dataframe and drop the original columns.

In [35]:
sample_df = pd.concat([sample_df, proto_onehot_df, state_onehot_df, service_onehot_df, ip_onehot_df], axis=1)
sample_df = sample_df.drop(['srcip','dstip','proto','state','service'], axis=1)
sample_df.head()

,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label,...,vrrp,wb-expak,wb-mon,wsn,xnet,xns-idp,xtp,zero,ACC,CLO,CON,ECO,ECR,FIN,INT,REQ,URH,-,dhcp,dns,ftp,ftp-data,http,irc,pop3,radius,smtp,snmp,ssh,ssl,S10.40.0.0/16,S149.171.126.0/24,S175.45.176.0/24,S59.166.0.0/24,Sothers,D10.40.0.0/16,D149.171.126.0/24,D175.45.176.0/24,D59.166.0.0/24,Dothers
87823,1.833348e-07,0.000015,0.000000,0.996078,0.000000,0.000000,0.000000,0.013805,0.000000,0.000196,0.000000,0.0,0.0,0.000000,0.000000,0.066489,0.000000,0.0,0.0,0.000000,0.000000,1.833212e-07,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.0,0.046512,0.048780,0.024390,0.102041,0.028571,0.030303,0.027027,2,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
417228,7.750228e-04,0.000083,0.000161,0.121569,0.114173,0.000785,0.000730,0.000035,0.017908,0.001373,0.001276,1.0,1.0,0.837667,0.337233,0.053856,0.114888,0.0,0.0,0.000248,0.000012,5.914609e-05,7.078510e-05,0.000546,0.000882,0.000151,0.0,0.000000,0.0,0.0,0.0,0.418605,0.585366,0.512195,0.367347,0.085714,0.090909,0.081081,9,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
363125,1.761681e-05,0.000011,0.000012,0.121569,0.114173,0.000000,0.000000,0.000105,0.031915,0.000196,0.000182,0.0,0.0,0.000000,0.000000,0.048537,0.060503,0.0,0.0,0.000000,0.000000,8.332782e-08,1.601333e-07,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.046512,0.073171,0.097561,0.000000,0.000000,0.000000,0.000000,9,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
145077,8.333399e-08,0.000008,0.000000,0.996078,0.000000,0.000000,0.000000,0.017312,0.000000,0.000196,0.000000,0.0,0.0,0.000000,0.000000,0.037899,0.000000,0.0,0.0,0.000000,0.000000,8.332782e-08,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.0,0.139535,0.146341,0.073171,0.102041,0.085714,0.090909,0.081081,5,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
436433,4.241017e-03,0.000653,0.000022,0.121569,0.114173,0.000785,0.000182,0.000049,0.000398,0.001373,0.000547,1.0,1.0,0.668125,0.216809,0.424202,0.036030,0.0,0.0,0.001384,0.000145,3.262079e-04,1.001834e-03,0.001445,0.002692,0.000174,0.0,0.000000,0.0,0.0,0.0,0.046512,0.048780,0.512195,0.285714,0.057143,0.060606,0.297297,9,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [36]:
sample_df.columns.values

array(['dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss',
       'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb',
       'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len',
       'Sjit', 'Djit', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst',
       'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat', 'label',
       'last-start', '3pc', 'a/n', 'aes-sp3-d', 'any', 'argus', 'aris',
       'arp', 'ax.25', 'bbn-rcc', 'bna', 'br-sat-mon', 'cbt', 'cftp',
       'chaos', 'compaq-peer', 'cphb', 'cpnx', 'crtp', 'crudp', 'dcn',
       'ddp', 'ddx', 'dgp', 'egp', 'eigrp', 'emcon', 'encap', 'etherip',
       'fc', 'fire', 'ggp', 'gmtp', 'gre', 'hmp', 'i-nlsp', 'iatp', 'ib',
       'icmp', 'idpr', 'idpr-cmtp', 'idrp', 'ifmp', 'igp', 'il', 'ip',
       'ipcomp', 'ipcv', 'ipip', 'iplt'

In [37]:
# put attack categories and labels after all features

cols = sample_df.columns.tolist()
cols = cols[:38] + cols[40:] + cols[38:40]
sample_df = sample_df[cols]
sample_df.head()

,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,last-start,3pc,...,wb-mon,wsn,xnet,xns-idp,xtp,zero,ACC,CLO,CON,ECO,ECR,FIN,INT,REQ,URH,-,dhcp,dns,ftp,ftp-data,http,irc,pop3,radius,smtp,snmp,ssh,ssl,S10.40.0.0/16,S149.171.126.0/24,S175.45.176.0/24,S59.166.0.0/24,Sothers,D10.40.0.0/16,D149.171.126.0/24,D175.45.176.0/24,D59.166.0.0/24,Dothers,attack_cat,label
87823,1.833348e-07,0.000015,0.000000,0.996078,0.000000,0.000000,0.000000,0.013805,0.000000,0.000196,0.000000,0.0,0.0,0.000000,0.000000,0.066489,0.000000,0.0,0.0,0.000000,0.000000,1.833212e-07,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.0,0.046512,0.048780,0.024390,0.102041,0.028571,0.030303,0.027027,0.0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,2,1
417228,7.750228e-04,0.000083,0.000161,0.121569,0.114173,0.000785,0.000730,0.000035,0.017908,0.001373,0.001276,1.0,1.0,0.837667,0.337233,0.053856,0.114888,0.0,0.0,0.000248,0.000012,5.914609e-05,7.078510e-05,0.000546,0.000882,0.000151,0.0,0.000000,0.0,0.0,0.0,0.418605,0.585366,0.512195,0.367347,0.085714,0.090909,0.081081,0.0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,9,0
363125,1.761681e-05,0.000011,0.000012,0.121569,0.114173,0.000000,0.000000,0.000105,0.031915,0.000196,0.000182,0.0,0.0,0.000000,0.000000,0.048537,0.060503,0.0,0.0,0.000000,0.000000,8.332782e-08,1.601333e-07,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.046512,0.073171,0.097561,0.000000,0.000000,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,9,0
145077,8.333399e-08,0.000008,0.000000,0.996078,0.000000,0.000000,0.000000,0.017312,0.000000,0.000196,0.000000,0.0,0.0,0.000000,0.000000,0.037899,0.000000,0.0,0.0,0.000000,0.000000,8.332782e-08,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.0,0.139535,0.146341,0.073171,0.102041,0.085714,0.090909,0.081081,0.0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,5,1
436433,4.241017e-03,0.000653,0.000022,0.121569,0.114173,0.000785,0.000182,0.000049,0.000398,0.001373,0.000547,1.0,1.0,0.668125,0.216809,0.424202,0.036030,0.0,0.0,0.001384,0.000145,3.262079e-04,1.001834e-03,0.001445,0.002692,0.000174,0.0,0.000000,0.0,0.0,0.0,0.046512,0.048780,0.512195,0.285714,0.057143,0.060606,0.297297,0.0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,9,0


---
## Analysis methods & results

### Binary classification : neural network
The classification separates the attacks from the normals.

In [0]:
labels = sample_df['label']
features = sample_df.iloc[:,:len(sample_df.columns)-2]

X = features
y = labels

In [39]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(8, activation='relu', input_shape=(len(sample_df.columns)-2,)))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
                   
model.fit(features, labels,epochs=10, batch_size=1, verbose=1)

Using TensorFlow backend.


Epoch 1/10
52215/52215 [==============================] - 53s 1ms/step - loss: 0.0354 - accuracy: 0.9897
Epoch 2/10
52215/52215 [==============================] - 53s 1ms/step - loss: 0.0211 - accuracy: 0.9957
Epoch 3/10
52215/52215 [==============================] - 53s 1ms/step - loss: 0.0195 - accuracy: 0.9958
Epoch 4/10
52215/52215 [==============================] - 52s 1ms/step - loss: 0.0199 - accuracy: 0.9955
Epoch 5/10
52215/52215 [==============================] - 53s 1ms/step - loss: 0.0191 - accuracy: 0.9957
Epoch 6/10
52215/52215 [==============================] - 52s 999us/step - loss: 0.0190 - accuracy: 0.9957
Epoch 7/10
52215/52215 [==============================] - 52s 998us/step - loss: 0.0193 - accuracy: 0.9956
Epoch 8/10
52215/52215 [==============================] - 52s 997us/step - loss: 0.0188 - accuracy: 0.9958
Epoch 9/10
52215/52215 [==============================] - 52s 997us/step - loss: 0.0185 - accuracy: 0.9958
Epoch 10/10
52215/52215 [======================

Now let's try neural network with dimensionally-reduced data.

In [0]:
from sklearn.decomposition import PCA

pca = PCA(n_components=20)
features_pca = pca.fit_transform(features)

In [41]:
model = Sequential()
model.add(Dense(8, activation='relu', input_shape=(20,)))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
                   
model.fit(features_pca, labels,epochs=8, batch_size=1, verbose=1)

Epoch 1/8
52215/52215 [==============================] - 49s 935us/step - loss: 0.0360 - accuracy: 0.9913
Epoch 2/8
52215/52215 [==============================] - 48s 923us/step - loss: 0.0216 - accuracy: 0.9955
Epoch 3/8
52215/52215 [==============================] - 49s 935us/step - loss: 0.0208 - accuracy: 0.9955
Epoch 4/8
52215/52215 [==============================] - 48s 928us/step - loss: 0.0210 - accuracy: 0.9953
Epoch 5/8
52215/52215 [==============================] - 49s 930us/step - loss: 0.0208 - accuracy: 0.9952
Epoch 6/8
52215/52215 [==============================] - 49s 938us/step - loss: 0.0196 - accuracy: 0.9957
Epoch 7/8
52215/52215 [==============================] - 50s 963us/step - loss: 0.0197 - accuracy: 0.9956
Epoch 8/8
52215/52215 [==============================] - 48s 928us/step - loss: 0.0188 - accuracy: 0.9959


### Multiclass classification : decision tree
The classification separates the data into 10 attack categories.

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

labels = sample_df['attack_cat']
features = sample_df.iloc[:,:len(sample_df.columns.values)-2]

X = features
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [43]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dtc = DecisionTreeClassifier()
z = dtc.fit(X_train,y_train)

acc = accuracy_score(y_test,z.predict(X_test))
print('Accuracy of decision tree: {}%'.format(round(acc*100, 2)))

Accuracy of decision tree: 89.51%


### Multiclass classification : KNN
The classification separates the data into 10 attack categories.

In [44]:
from sklearn.neighbors import KNeighborsClassifier
from time import time

knn = KNeighborsClassifier()
start = time()
zz = knn.fit(X_train,y_train)

acc1 = accuracy_score(y_test,zz.predict(X_test))
print('Accuracy of KNN : {}%'.format(round(acc1*100, 2)))
print('took', time() - start, 'seconds.')

Accuracy of KNN : 88.07%
took 19.485621690750122 seconds.


In [45]:
pca = PCA(n_components=20)
Xtrain_pca = pca.fit_transform(X_train)
Xtest_pca = pca.fit_transform(X_test)

start = time()
zzz = knn.fit(Xtrain_pca,y_train)
acc2 = accuracy_score(y_test,zzz.predict(Xtest_pca))
print('Accuracy of KNN with dimensionally-reduced data: {}%'.format(round(acc2*100, 2)))
print('took', time() - start, 'seconds.')

Accuracy of KNN with dimensionally-reduced data: 86.79%
took 1.2459187507629395 seconds.


According to the results, applying PCA may lower the accuracy but the efficiency improved significantly especially when the dataset is large.